In [1]:
import json
from bs4 import BeautifulSoup 
import requests 
import re 
import os
from UpstreamCommitsFinder import UpstreamCommitsFinder

In [2]:
cache = True
filename = "../linux-commits-2023-11-12_random-filtered-1.json"

finder = UpstreamCommitsFinder(filename)
#finder.get_all(cache=True)
finder.get_branch("6.1")
finder.print_stats()

https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.1
 > Upstream commits found: 0
 > Upstream commits match with annotated commits: 0
https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.1.1
 > Upstream commits found: 24
 > Upstream commits match with annotated commits: 1
https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.1.10
 > Upstream commits found: 3
 > Upstream commits match with annotated commits: 0
https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.1.100
 > Upstream commits found: 41
 > Upstream commits match with annotated commits: 0
https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.1.101
 > Upstream commits found: 15
 > Upstream commits match with annotated commits: 0
https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.1.102
 > Upstream commits found: 15
 > Upstream commits match with annotated commits: 0
https://cdn.kernel.org/pub/linux/kernel/v6.x/ChangeLog-6.1.103
 > Upstream commits found: 125
 > Upstream commits match with annotated comm